In [34]:
import mido
import pandas as pd

# Function to convert MIDI note number to frequency
def midi_note_to_frequency(note_number):
    # MIDI note number to frequency conversion formula
    return 440.0 * (2.0 ** ((note_number - 69) / 12.0))

# Load MIDI file
midi_file = mido.MidiFile('dafengchui.mid')

# Create a list to hold note information
notes_list = []

# Initialize a variable to keep track of the current time
current_time = 0

# Dictionary to keep track of note start times
note_start_times = {}

# Iterate over all messages in the MIDI file
for msg in midi_file:
    # Update the current time
    current_time += msg.time
    if msg.type == 'note_on' and msg.velocity > 0:
        # Note on message, start of a note
        note_start_times[msg.note] = current_time
    elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
        # Note off message, end of a note
        if msg.note in note_start_times:
            note_start_time = note_start_times.pop(msg.note)
            note_end_time = current_time
            frequency = midi_note_to_frequency(msg.note)
            notes_list.append([note_start_time, note_end_time, frequency])

# Create a DataFrame with the notes list
df = pd.DataFrame(notes_list, columns=['start_time', 'end_time', 'frequency'])

# Display the DataFrame
print(df)

      start_time    end_time   frequency
0       0.000000    0.568333  246.941651
1       0.000000    0.568333  293.664768
2       0.000000    0.568333  369.994423
3       0.600000    0.788333  246.941651
4       0.600000    0.788333  293.664768
...          ...         ...         ...
1702  262.000000  262.378333  184.997211
1703  262.463333  264.796667  164.813778
1704  262.531667  264.796667  220.000000
1705  262.598333  264.796667  277.182631
1706  262.400000  264.798333  138.591315

[1707 rows x 3 columns]


In [35]:
# add a column of just "@" as the leftmost column
df.insert(0, ' ', '@')

# add a column of just "SineEnv" as the fourth column
df.insert(3, 'SineEnv', 'SineEnv')

# add a column called "amplitude" of just "0.3" as the fifth column
df.insert(4, 'amplitude', 0.3)

# add a column called "attack_time" of just "0.034" as the seventh column
df.insert(6, 'attack_time', 0.034)

# add a column called "release_time" of just "0.458" as the eighth column
df.insert(7, 'release_time', 0.458)

# add a column called "pan" of just "0" as the ninth column
df.insert(8, 'pan', 0)

# add columns pianoKeyX pianoKeyY pianoKeyWidth pianoKeyHeight of values 2 0 116 296
df.insert(9, 'pianoKeyX', 2)
df.insert(10, 'pianoKeyY', 0)
df.insert(11, 'pianoKeyWidth', 116)
df.insert(12, 'pianoKeyHeight', 296)

In [36]:
df

,,start_time,end_time,SineEnv,amplitude,frequency,attack_time,release_time,pan,pianoKeyX,pianoKeyY,pianoKeyWidth,pianoKeyHeight
0,@,0.000000,0.568333,SineEnv,0.3,246.941651,0.034,0.458,0,2,0,116,296
1,@,0.000000,0.568333,SineEnv,0.3,293.664768,0.034,0.458,0,2,0,116,296
2,@,0.000000,0.568333,SineEnv,0.3,369.994423,0.034,0.458,0,2,0,116,296
3,@,0.600000,0.788333,SineEnv,0.3,246.941651,0.034,0.458,0,2,0,116,296
4,@,0.600000,0.788333,SineEnv,0.3,293.664768,0.034,0.458,0,2,0,116,296
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,@,262.000000,262.378333,SineEnv,0.3,184.997211,0.034,0.458,0,2,0,116,296
1703,@,262.463333,264.796667,SineEnv,0.3,164.813778,0.034,0.458,0,2,0,116,296
1704,@,262.531667,264.796667,SineEnv,0.3,220.000000,0.034,0.458,0,2,0,116,296
1705,@,262.598333,264.796667,SineEnv,0.3,277.182631,0.034,0.458,0,2,0,116,296


In [37]:
# replace end_time with duration without changing the order of the columns
duration = df['end_time'] - df['start_time']
df = df.drop(columns=['end_time'])
df.insert(2, 'duration', duration)

df

,,start_time,duration,SineEnv,amplitude,frequency,attack_time,release_time,pan,pianoKeyX,pianoKeyY,pianoKeyWidth,pianoKeyHeight
0,@,0.000000,0.568333,SineEnv,0.3,246.941651,0.034,0.458,0,2,0,116,296
1,@,0.000000,0.568333,SineEnv,0.3,293.664768,0.034,0.458,0,2,0,116,296
2,@,0.000000,0.568333,SineEnv,0.3,369.994423,0.034,0.458,0,2,0,116,296
3,@,0.600000,0.188333,SineEnv,0.3,246.941651,0.034,0.458,0,2,0,116,296
4,@,0.600000,0.188333,SineEnv,0.3,293.664768,0.034,0.458,0,2,0,116,296
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,@,262.000000,0.378333,SineEnv,0.3,184.997211,0.034,0.458,0,2,0,116,296
1703,@,262.463333,2.333333,SineEnv,0.3,164.813778,0.034,0.458,0,2,0,116,296
1704,@,262.531667,2.265000,SineEnv,0.3,220.000000,0.034,0.458,0,2,0,116,296
1705,@,262.598333,2.198333,SineEnv,0.3,277.182631,0.034,0.458,0,2,0,116,296


In [38]:
# convert dataframe to a space separated file
df.to_csv('bin/SineEnv_Piano-data/dafengchui.synthSequence', sep=' ', index=False, header=False)

In [ ]:
# create dict mapping frequency to note
note_dict = {}
for i in range(128):
    note_dict[midi_note_to_frequency(i)] = i

    